In [1]:
import numpy as np
import config

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
from tensorflow import keras

def create_model():
    print('Creating new model...', end='\r')
    
    VGG_base = keras.applications.vgg16.VGG16(include_top=False, input_shape=config.input_shape)
    for layer in VGG_base.layers[:-2]:
        layer.trainable = False
    
    input_layer = keras.Input(shape=config.input_shape)
    VGG = keras.applications.vgg16.preprocess_input(input_layer)
    VGG = VGG_base(VGG)
    VGG = keras.layers.GlobalAveragePooling2D()(VGG)
    
    net = keras.layers.Dense(units=4096)(VGG)
    net = keras.layers.BatchNormalization()(net)
    net = keras.layers.Activation(keras.activations.relu)(net)
    net = keras.layers.Dropout(0.3)(net)
    
    net = keras.layers.Dense(units=2048)(net)
    net = keras.layers.BatchNormalization()(net)
    net = keras.layers.Activation(keras.activations.relu)(net)
    net = keras.layers.Dropout(0.3)(net)
    
    output_layer = keras.layers.Dense(units=config.output_shape, activation='softmax')(net)
    
    model=keras.Model(inputs=[input_layer],outputs=[output_layer])
    model.compile(
        loss='categorical_crossentropy',
        optimizer=keras.optimizers.SGD(
            learning_rate=3e-4,
            momentum=0.9,
            decay=1e-5,
            nesterov=True
        ),
        metrics=[
            keras.metrics.CategoricalAccuracy(),
            keras.metrics.Precision(),
            keras.metrics.Recall()
        ]
    )
    print('Successfully created new model.')
    return model

def load_model(dir):
    print('Loading model from %s'%dir, end='\r')
    model = keras.models.load_model(dir)
    print('Successfully loaded model from %s'%dir)
    return model

def save_model(dir, model):
    print('Saving model to %s'%dir, end='\r')
    model.save(dir)
    print('Successfully saved model to %s'%dir)

def save_plot(dir, model):
    print('Saving model plot to %s'%dir, end='\r')
    keras.utils.plot_model(model,dir, show_shapes=True, expand_nested=True)
    print('Successfully saved model plot to %s'%dir)

def save_summary(dir, model):
    print('Saving model summary to %s'%dir, end='\r')
    with open(dir, 'w') as f:
        model.summary(print_fn=lambda x:f.write(x + '\n'), expand_nested=True)
    print('Successfully saved model summary to %s'%dir)

In [3]:
augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip(mode='horizontal'),
        keras.layers.RandomZoom(height_factor=(-0.1,0.1)),
        keras.layers.RandomTranslation(height_factor=(-0.1,0.1), width_factor=(-0.1,0.1)),
        keras.layers.RandomRotation(factor = (0.1))
    ]
)

In [4]:
def create_index(source):
    return (source, os.listdir(source))

def generator(source, index, num_batch, data_augmentation=True):
    while(True):
        np.random.shuffle(index)
        for i in range(num_batch):
            X, y = np.load( os.path.join(source, index[i]), allow_pickle=True )
            X = np.asarray( list(X), 'float32' )
            if(data_augmentation):
                X = augmentation(X)
            y = np.asarray( list(y), 'float32' )
            yield(X, y)    

def create_generator(index, data_augmentation=True):
    source = index[0]
    index = index[1]
    num_batch = len(index)
    gen = generator(source, index, num_batch, data_augmentation)
    return gen, num_batch

In [10]:
model_name = 'v2.2-vgg16'

#### Training starts here.

In [ ]:
# model = create_model()
model = load_model( os.path.join(config.dir_model_saved, '%s/'%model_name) )

# save_model(os.path.join(config.dir_model_saved, '%s/'%model_name), model)
# save_plot( os.path.join(config.dir_model_plot, '%s-plot.png'%model_name), model )
# save_summary( os.path.join(config.dir_model_summary, '%s-summary.txt'%model_name), model )

In [ ]:
train_index = create_index(config.train_batch)
val_index = create_index(config.val_batch)

train_gen, train_num_batch = create_generator(train_index)
val_gen, val_num_batch = create_generator(val_index, data_augmentation=False)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath=os.path.join(config.dir_model_saved, '%s/'%model_name), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

history = model.fit(
    x = train_gen,
    steps_per_epoch = train_num_batch,
    validation_data = val_gen,
    validation_steps = val_num_batch,
    epochs = 30,
    verbose = 1,
    callbacks = callbacks
)

#### Testing and evaluation starts here.

In [11]:
test_index = create_index(config.test_batch)
test_gen, test_num_batch = create_generator(test_index, data_augmentation=False)

In [12]:
test_model = load_model( os.path.join(config.dir_model_saved, '%s/'%model_name) )
result = test_model.evaluate(
    x = test_gen,
    steps = test_num_batch-1,
    verbose = 1
)

print()
print('Accuracy  : %.2f%%'%( result[1]*100 ))
print('Precision : %.2f%%'%( result[2]*100 ))
print('Recall    : %.2f%%'%( result[3]*100 ))

Successfully loaded model from ../Resources/model_saved/v2.2-vgg16/
224/224 [==============================] - 72s 317ms/step - loss: 1.3570 - categorical_accuracy: 0.5635 - precision_1: 0.6320 - recall_1: 0.5029

Accuracy  : 56.35%
Precision : 63.20%
Recall    : 50.29%


In [13]:
y_truth = np.array([])
y_predict = np.array([])

for i in range(test_num_batch):
    X, y = next(test_gen)
    prediction = test_model(X).numpy()
    y_truth = np.hstack( (y_truth, np.argmax(y, axis=1)) )
    y_predict = np.hstack( (y_predict, np.argmax(prediction, axis=1)) )

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_truth, y_predict)
print(cm)

[[ 489   46  101   33  127   69   93]
 [  16   74    7    3    3    6    2]
 [ 149   33  359   22  146  203  112]
 [  79   27   42 1325   35  149  117]
 [ 164   41  193   85  434   83  247]
 [  41    9   54   25    7  670   25]
 [ 102   12  112   75  141   96  695]]


In [14]:
for i in config.facial_expression:
    print("%-20s : %.2f%%"%(config.facial_expression[i], cm[i][i]*100/cm[i].sum()))

angry                : 51.04%
disgust              : 66.67%
fear                 : 35.06%
happy                : 74.69%
sad                  : 34.80%
surprise             : 80.63%
neutral              : 56.37%
